In [1]:
#Install elastic search
!pip install elasticsearch


In [4]:
!pip install -q pandas openai

In [7]:
# To search documents 

#Import Elascticsearch and helpers from  elasticsearch

from elasticsearch import Elasticsearch,helpers
import json

es = Elasticsearch(['http://localhost:9200'])
es.ping()   #connection testing
res = es.search(index="github_issues", body={"query": {"match_all": {}}})

# Extract the total hit count
print("Got %d Hits:" % res['hits']['total']['value'])

# Extract hits from the response
hits = res.get('hits', {})

# Pretty print the hits
print(json.dumps(hits, indent=2))


Got 6355 Hits:
{
  "total": {
    "value": 6355,
    "relation": "eq"
  },
  "max_score": 1.0,
  "hits": [
    {
      "_index": "github_issues",
      "_id": "96LFTZMBc2Nzseb3Zt5W",
      "_score": 1.0,
      "_source": {
        "_type": "issue",
        "_repo": "langchain",
        "_issueNumber": "28250",
        "_title": "DOC: What is the maximum chunk size returned from SemanticChunker.split_documents()",
        "_createdAt": "2024-11-21T06:50:49Z",
        "_closedAt": "2024-12-31T00:36:30Z",
        "_state": "open",
        "_body": "### URL\n\n_No response_\n\n### Checklist\n\n- [X] I added a very descriptive title to this issue.\n- [X] I included a link to the documentation page I am referring to (if applicable).\n\n### Issue with current documentation:\n\nI need to know on the maximum chunk size that can be return from SemanticChunker.split_documents() for large documents.\r\nCan it be more than 8k token as i need to send the chunk for embedding, chunk with more than 8k 

In [10]:
# Define model and embedding function
from openai import OpenAI
EMBEDDING_MODEL = "text-embedding-ada-002"
client = OpenAI(api_key="sk-proj-Xp8GTxnJ8yrqA2jqoF3qT3BlbkFJRnlKYZJBisoEiyw1eGFz")
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding


In [13]:
# Pretty print function for GitHub Issues

def pretty_response_Issue(response):
    for hit in response['hits']['hits']:
        type = hit['_source']['_type']
        repo =hit['_source']['_repo']
        issueNumber = hit['_source']['_issueNumber']
        title = hit['_source']['_title']
        createdAt = hit['_source']['_createdAt']
        closedAt = hit['_source']['_closedAt']
        state = hit['_source']['_state']
        body = hit['_source']['_body']
        
        pretty_output = (
            f"\nType: {type}\nRepo:{repo}\nIssue Number: {issueNumber}\nTitle: {title}\nCreated At: {createdAt}\nClosed At: {closedAt}\n"
            f"State: {state}\nBody: {body}\n"
        )
        print(pretty_output)

<div class="alert alert-block alert-success">

## <center>Part-II (2) Use vector embeddings, score, and semantic search to identify and list the Top 5 most similar issues for every repo listed above.</center>

    

</div>

In [ ]:
import json

question = 'find the top 5 similar issues that are in the index for elasticsearch'

question_embedding = get_embedding(question, model=EMBEDDING_MODEL)
response = es.search(
    index="github_issues",
    knn={
        "field":"GitHub_Issue_vector",
        "query_vector": hits['hits'][5]['_source']["GitHub_Issue_vector"],
        "k":50,
        "num_candidates": 10000
    }
)
responses = []
for response in response["hits"]["hits"]:
    response_obj={
        "_score":response["_score"],
        "_type" :response["_source"]["_type"],
        "_repo" :response["_source"]["_repo"],
        "_issueNumber":response["_source"]["_issueNumber"],
        "title":response["_source"]["_title"],
        "_createdAt":response["_source"]["_createdAt"],
        "_closedAt":response["_source"]["_closedAt"],
        "_body":response["_source"]["_body"],
    }
    responses.append(response_obj)

In [22]:
for response in responses[:5]:
    print(response)
    print()

{'_score': 1.0, '_type': 'issue', '_repo': 'langchain', '_issueNumber': '28243', 'title': 'infra: remove stale dockerfiles from repo', '_createdAt': '2024-11-20T23:56:08Z', '_closedAt': '2024-11-21T00:05:01Z', '_body': "deleting the following docker things from monorepo. they aren't currently usable because of old dependencies, and I'd rather avoid people using them / having to maintain them\r\n\r\n- /docker\r\n  - this folder has a compose file that spins up postgres,pgvector (separate from postgres and very stale version),mongo instance with default user/password that we've gotten security pings about before. not worth having\r\n  - also spins up a custom dockerfile with onttotext/graphdb - not even sure what that is\r\n- /libs/langchain/dockerfile + dev.dockerfile\r\n  - super old poetry version, doesn't implement the right thing anymore\r\n- .github/workflows/_release_docker.yml, langchain_release_docker.yml\r\n  - not used anymore, not worth having an alternate release path\r\n"}


## Sanity Test:

Execute following commands from the command window/terminal:

**To verify the index got created:**
- curl -XGET "http://localhost:9200/_cat/indices?v"

**To check the structure of index:**
- curl -X GET "localhost:9200/github_issues?pretty"

**To check the stats of the index:**

- curl -X GET "localhost:9200/github_issues/_stats?pretty"

**To dump the documents in the index to stdout:**
- curl -XGET "http://localhost:9200/github_issues/_search?pretty=true"

**To dump the mapping of  the index to stdout:**
- curl -XGET "localhost:9200/github_issues/_mapping?pretty"


## How to delete  the index?
**To delete the index you created:**
- curl -XDELETE "http://localhost:9200/github_issues?pretty"

